# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [16]:
#!cat publications.csv

## Import pandas

We are using the very handy pandas library for dataframes.

In [8]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [43]:
conv = {'publi_id': lambda x: str(x), 'page': lambda x: str(x), 'volume': lambda x: str(x), 'arxiv_id': lambda x: str(x)}
publications = pd.read_csv("publications.csv", sep=";", header=0, converters=conv)#, engine ='python')
publications

,pub_date,title,authors,venue,volume,page,excerpt,url_slug,paper_url,arxiv_id,tags
0,2013-06-04,"Hysteresis, phase transitions, and dangerous t...","C. Duclut, S. Backhaus, and M. Chertkov",Phys. Rev. E,87,62802,The majority of dynamical studies in power sys...,duclut2013hysteresis,https://link.aps.org/doi/10.1103/PhysRevE.87.0...,1212.0252,NaN
1,2016-09-02,Langevin Equations for Reaction-Diffusion Proc...,"F. Benitez, C. Duclut, H. Chaté, B. Delamotte,...",Phys. Rev. Lett.,117,100601,For reaction-diffusion processes with at most ...,benitez2016langevin,https://link.aps.org/doi/10.1103/PhysRevLett.1...,1604.06740,statisticalPhysics
2,2017-01-05,Frequency regulators for the nonperturbative r...,"C. Duclut, and B. Delamotte",Phys. Rev. E,95,12107,We derive the necessary conditions for impleme...,duclut2017frequency,http://link.aps.org/doi/10.1103/PhysRevE.95.01...,1611.07301,"statisticalPhysics, phaseTransition, renormali..."
3,2017-07-27,Nonuniversality in the erosion of tilted lands...,"C. Duclut, and B. Delamotte",Phys. Rev. E,96,12149,The anisotropic model for landscapes erosion p...,duclut2017nonuniversality,http://link.aps.org/doi/10.1103/PhysRevE.96.01...,1705.05294,"statisticalPhysics, phaseTransition, renormali..."
4,2019-09-24,Fluid pumping and active flexoelectricity can ...,"C. Duclut, N. Sarkar, J. Prost, and F. Jülicher",Proc. Natl. Acad. Sci. U.S.A.,116,19264,We discuss the physical mechanisms that promot...,duclut2019fluid,http://www.pnas.org/lookup/doi/10.1073/pnas.19...,1905.08023,"tissues, electrohydraulics, hydraulics"
5,2021-01-29,Nonequilibrium polarity-induced chemotaxis: Em...,"S. Mahdisoltani, R. Ben Alì Zinati, C. Duclut,...",Phys. Rev. Research,3,13100,A generically observed mechanism that drives t...,mahdisoltani2021nonequilibrium,https://link.aps.org/doi/10.1103/PhysRevResear...,1911.08115,"statisticalPhysics, phaseTransition, renormali..."
6,2021-05-11,Hydraulic and electric control of cell spheroids,"C. Duclut, J. Prost, and F. Jülicher",Proc. Natl. Acad. Sci. U.S.A.,118,e2021972118,We use a theoretical approach to examine the e...,duclut2021hydraulic,http://www.pnas.org/lookup/doi/10.1073/pnas.20...,2011.03735,"tissues, electrohydraulics, hydraulics"
7,2021-09-27,Nonlinear rheology of cellular networks,"C. Duclut, J. Paijmans, M. M. Inamdar, C. D. M...",Cells & Development,168,203746,Morphogenesis depends crucially on the complex...,duclut2021nonlinear,https://linkinghub.elsevier.com/retrieve/pii/S...,2103.16462,"tissues, mechanics, rheology, vertexModel"
8,2022-03-11,Coarse-grained curvature tensor on polygonal s...,"C. Duclut, A. Amiri, J. Paijmans, and F. Jülicher",SciPost Phys. Core,5,11,"Using concepts from integral geometry, we prop...",duclut2022coarse-grained,https://scipost.org/10.21468/SciPostPhysCore.5...,2104.07988,"integralGeometry, tissues"
9,2022-05-26,Passive odd viscoelasticity,"R. Lier, J. Armas, S. Bo, C. Duclut, F. Jülich...",Phys. Rev. E,105,54607,Active chiral viscoelastic materials exhibit e...,lier2022passive,https://link.aps.org/doi/10.1103/PhysRevE.105....,2109.06606,"oddElasticity, oddViscosity, rheology, thermod..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [44]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [45]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
#     md_filename = item.url_slug + ".md"
#     html_filename = item.url_slug
    year = item.pub_date[:4]
    
    pub_number = str(row+1)
    
    if (item.venue=='preprint'):
        citation = '"' + item.title + '", ' + item.authors + \
        ', <i>arXiv:' + item.arxiv_id + '</i>' + \
        ' (' + str(year) + ').'
    elif (item.venue=='bioRxiv preprint'):
        citation = '"' + item.title + '", ' + item.authors + \
        ', <i>bioRxiv:' + item.arxiv_id + '</i>' + \
        ' (' + str(year) + ').'
    else:
        citation = '"' + item.title + '", ' + item.authors + \
        ', <i>' + item.venue + '</i> <b>' + \
        str(item.volume) + '</b>, ' + str(item.page) \
        + ' (' + str(year) + ').'
        
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    md += "\nurlslug: '" + item.url_slug + "'"
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nauthors: '" + item.authors + "'"
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    md += "\nvolume: '" + str(item.volume) + "'"
    
    md += "\npage: '" + str(item.page) + "'"
    
    if (item.paper_url=='/'):
        if (item.venue=='bioRxiv preprint'):
            md += "\npaperurl: 'https://www.biorxiv.org/content/10.1101/" + item.arxiv_id + "'"
        else:    
            md += "\npaperurl: 'https://arxiv.org/abs/" + item.arxiv_id + "'"
    else:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\narxivid: '" + item.arxiv_id + "'"
        
    md += "\npubnumber: " + pub_number
    
    md += "\ncitation: '" + html_escape(citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
#     if len(str(item.excerpt)) > 5:
#         md += "\n" + html_escape(item.excerpt) + "\n"
    
#     if len(str(item.paper_url)) > 5:
#         md += "\n[Download paper here](" + item.paper_url + ")\n" 

#     md += '\nRecommended citation:"' + item.title + '", ' + item.authors + \
#     ', <i>' + item.venue + '</i>, <b>' + \
#     str(item.volume) + '</b>, ' + str(item.page) \
#     + ' (' + str(year) + '). ' + [item.paper_url](item.paper_url)

    if (item.venue == 'preprint'):
        md += '\n[arXiv version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](https://arxiv.org/abs/' + str(item.arxiv_id) + ")\n" 
        md += '[[pdf] <i class="fa fa-download fa-xs" aria-hidden="true"></i>](http://charlieduclut.github.io/files/' + item.url_slug + ".pdf)\n<br/>"
    elif (item.venue=='bioRxiv preprint'):
        md += '\n[bioRxiv version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](https://www.biorxiv.org/content/10.1101/' + str(item.arxiv_id) + ")\n" 
        md += '[[pdf] <i class="fa fa-download fa-xs" aria-hidden="true"></i>](http://charlieduclut.github.io/files/' + item.url_slug + ".pdf)\n<br/>"       
    else:
        md += '\n[Published version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](' + item.paper_url + ")\n" 
        md += '[[pdf] <i class="fa fa-download fa-xs" aria-hidden="true"></i>](http://charlieduclut.github.io/files/' + item.url_slug + ".pdf)\n<br/>"
        md += '\n[arXiv version <i class="fa fa-external-link-alt fa-xs" aria-hidden="true"></i>](https://arxiv.org/abs/' + str(item.arxiv_id) + ")\n"      
     
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [46]:
!ls ../_publications/

2013-06-04-duclut2013hysteresis.md
2016-09-02-benitez2016langevin.md
2017-01-05-duclut2017frequency.md
2017-07-27-duclut2017nonuniversality.md
2019-09-24-duclut2019fluid.md
2021-01-29-mahdisoltani2021nonequilibrium.md
2021-04-16-duclut2021coarse-grained.md
2021-05-11-duclut2021hydraulic.md
2021-09-14-lier2021passive.md
2021-09-27-duclut2021nonlinear.md
2021-11-19-benalizinati2021stochastic.md
2021-11-19-duclut2021active.md
2022-01-06-benalizinati2022stochastic.md
2022-03-11-duclut2022coarse-grained.md
2022-03-23-duclut2022active.md
2022-05-25-lier2022lift.md
2022-05-26-lier2022passive.md
2022-09-30-tan2022emergent.md
2022-11-03-amiri2022random.md
2022-12-10-piscitello-gomez2022core.md


In [47]:
!cat ../_publications/2022-12-10-piscitello-gomez2022core.md

---
title: "Core PCP mutations affect short time mechanical properties but not tissue morphogenesis in the <i>Drosophila</i> pupal wing"
collection: publications
permalink: /publication/2022-12-10-piscitello-gomez2022core
urlslug: 'piscitello-gomez2022core'
excerpt: 'How morphogenetic movements are robustly coordinated in space and time is a fundamental open question in biology. We study this question using the wing of Drosophila melanogaster, an epithelial tissue that undergoes large-scale tissue flows during pupal stages. We showed previously (Etournay et al., 2015) that pupal wing morphogenesis involves both cellular behaviors that allow relaxation of mechanical tissue stress, as well as cellular behaviors that appear to be actively patterned. The core planar cell polarity (PCP) pathway influences morphogenetic cell movements in many other contexts, which suggests that it could globally pattern active cellular behaviors during pupal wing morphogenesis. We show here, however, that th

In [42]:
!cat ../_publications/2021-04-16-duclut2021coarse-grained.md

---
title: "Coarse-grained curvature tensor on polygonal surfaces"
collection: publications
permalink: /publication/2021-04-16-duclut2021coarse-grained
urlslug: 'duclut2021coarse-grained'
excerpt: 'Using concepts from integral geometry, we propose a definition for a local coarse-grained curvature tensor that is well-defined on polygonal surfaces. This coarse-grained curvature tensor shows fast convergence to the curvature tensor of smooth surfaces, capturing with accuracy not only the principal curvatures but also the principal directions of curvature. Thanks to the additivity of the integrated curvature tensor, coarse-graining procedures can be implemented to compute it over arbitrary patches of polygons. When computed for a closed surface, the integrated curvature tensor is identical to a rank-2 Minkowski tensor. We also provide an algorithm to extend an existing C++ package, that can be used to compute efficiently local curvature tensors on triangulated surfaces.'
date: 2021-04-16
a